# Algorithm 9: Triangle Multiplication Outgoing (AlphaFold3)

Triangle Multiplication with outgoing edges updates pair representation using triangle inequality constraints.

## Source Code Location
- **File**: `AF3-Ref-src/alphafold3-official/src/alphafold3/model/network/modules.py`

In [ ]:
import numpy as np
np.random.seed(42)

def layer_norm(x, eps=1e-5):
    mean = np.mean(x, axis=-1, keepdims=True)
    var = np.var(x, axis=-1, keepdims=True)
    return (x - mean) / np.sqrt(var + eps)

def sigmoid(x):
    return 1 / (1 + np.exp(-np.clip(x, -500, 500)))

In [ ]:
def triangle_multiplication_outgoing(z, c=128):
    """
    Triangle Multiplication with outgoing edges.
    
    Updates z_ij using z_ik and z_jk for all k.
    
    Args:
        z: Pair representation [N, N, c_z]
        c: Hidden dimension
    
    Returns:
        Update to pair representation [N, N, c_z]
    """
    N = z.shape[0]
    c_z = z.shape[-1]
    
    print(f"Triangle Multiplication (Outgoing)")
    print(f"="*50)
    print(f"Pair: [{N}, {N}, {c_z}]")
    
    # Layer norm
    z_norm = layer_norm(z)
    
    # Project to a and b with gating
    W_a = np.random.randn(c_z, c) * (c_z ** -0.5)
    W_ag = np.random.randn(c_z, c) * (c_z ** -0.5)
    W_b = np.random.randn(c_z, c) * (c_z ** -0.5)
    W_bg = np.random.randn(c_z, c) * (c_z ** -0.5)
    W_g = np.random.randn(c_z, c_z) * (c_z ** -0.5)
    
    # Gated projections
    a = sigmoid(z_norm @ W_ag) * (z_norm @ W_a)  # [N, N, c]
    b = sigmoid(z_norm @ W_bg) * (z_norm @ W_b)  # [N, N, c]
    g = sigmoid(z_norm @ W_g)  # [N, N, c_z]
    
    print(f"a: {a.shape}, b: {b.shape}")
    
    # Triangle multiplication: sum_k a_ik * b_jk (outgoing edges)
    # i ---> k <--- j
    output = np.einsum('ikc,jkc->ijc', a, b)  # [N, N, c]
    
    print(f"Triangle product: {output.shape}")
    
    # Layer norm and project back
    output = layer_norm(output)
    W_o = np.random.randn(c, c_z) * (c ** -0.5)
    output = (output @ W_o) * g  # Gating
    
    print(f"Output: {output.shape}")
    
    return output

In [ ]:
# Test
print("Test: Triangle Multiplication Outgoing")
print("="*60)

N = 32
c_z = 128

z = np.random.randn(N, N, c_z) * 0.1

output = triangle_multiplication_outgoing(z, c=64)

print(f"\nInput norm: {np.linalg.norm(z):.2f}")
print(f"Output norm: {np.linalg.norm(output):.2f}")
print(f"Output finite: {np.isfinite(output).all()}")

## Key Insights

1. **Outgoing Edges**: Uses edges i→k and j→k to update i→j
2. **Triangle Inequality**: Encodes geometric constraints
3. **Gated Projections**: sigmoid(gate) * projection for controlled info flow
4. **Sum over k**: Aggregates information from all possible third nodes